# 02 - Auto-sklearn 使用指南

> 🎯 学习目标：理解贝叶斯优化和元学习，掌握 Auto-sklearn 的高级特性

---

## 📚 本节内容

1. Auto-sklearn 原理简介
2. 基础使用方法
3. 贝叶斯优化深入
4. 元学习(Meta-learning)
5. 自动集成学习
6. 与 sklearn 生态集成

---

## 🧠 Auto-sklearn 核心原理

### 三大核心组件

1. **贝叶斯优化** (Bayesian Optimization)
   - 智能搜索超参数空间
   - 比随机搜索/网格搜索更高效

2. **元学习** (Meta-learning)
   - 从历史数据集学习经验
   - 快速找到好的起始点

3. **自动集成** (Ensemble)
   - 自动构建模型集成
   - 提升最终性能

---

## 1. 安装与导入

In [ ]:
# 安装 Auto-sklearn（首次运行）
# !pip install auto-sklearn
# 注意：Windows用户可能需要使用WSL或Docker

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

print("✅ 基础库导入成功")

---

## 2. 分类任务基础使用

### 2.1 加载数据

In [ ]:
from sklearn.datasets import load_digits

# 加载手写数字数据集
X, y = load_digits(return_X_y=True)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"训练集形状: {X_train.shape}")
print(f"测试集形状: {X_test.shape}")

### 2.2 使用 Auto-sklearn（基础版）

In [ ]:
import autosklearn.classification

# 创建分类器
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,  # 总时间预算（秒）
    per_run_time_limit=30,        # 单个模型训练时间限制
    seed=42
)

# 训练
print("开始训练（2分钟）...")
automl.fit(X_train, y_train)

print("\n✅ 训练完成！")

### 2.3 查看模型排行榜

In [ ]:
# 查看尝试过的所有模型
print("模型排行榜:")
print(automl.leaderboard())

### 2.4 预测与评估

In [ ]:
# 预测
y_pred = automl.predict(X_test)

# 评估
accuracy = accuracy_score(y_test, y_pred)
print(f"\n测试集准确率: {accuracy:.4f}")

print("\n分类报告:")
print(classification_report(y_test, y_pred))

### 2.5 查看最佳模型详情

In [ ]:
# 打印模型统计信息
print("Auto-sklearn 统计信息:")
print(automl.sprint_statistics())

# 查看最终集成模型
print("\n最终集成模型:")
print(automl.show_models())

---

## 3. 回归任务示例

### 3.1 加载回归数据

In [ ]:
from sklearn.datasets import load_diabetes

# 加载糖尿病数据集
X, y = load_diabetes(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"训练集形状: {X_train.shape}")

### 3.2 使用 Auto-sklearn 回归器

In [ ]:
import autosklearn.regression

# 创建回归器
automl_reg = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    seed=42
)

# 训练
print("开始训练回归模型...")
automl_reg.fit(X_train, y_train)

# 预测
y_pred = automl_reg.predict(X_test)

# 评估
from sklearn.metrics import r2_score, mean_squared_error
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"\nR² Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")

---

## 4. 高级特性

### 4.1 自定义指标

In [ ]:
from autosklearn.metrics import make_scorer, accuracy
from sklearn.metrics import f1_score

# 定义自定义指标
def f1_metric(y_true, y_pred):
    return f1_score(y_true, y_pred, average='weighted')

f1_scorer = make_scorer('f1_weighted', f1_metric)

# 使用自定义指标
automl_custom = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    metric=f1_scorer,
    seed=42
)

### 4.2 限制模型类型

In [ ]:
# 只使用特定的模型类型
automl_limited = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    include={
        'classifier': ['random_forest', 'extra_trees', 'gradient_boosting']
    },
    seed=42
)

# 或排除某些模型
automl_exclude = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    exclude={
        'classifier': ['k_nearest_neighbors']
    },
    seed=42
)

### 4.3 并行化

In [ ]:
# 使用多个CPU核心
automl_parallel = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    n_jobs=4,  # 使用4个核心
    seed=42
)

### 4.4 模型持久化

In [ ]:
import pickle

# 保存模型
with open('automl_model.pkl', 'wb') as f:
    pickle.dump(automl, f)

# 加载模型
with open('automl_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# 使用加载的模型预测
# predictions = loaded_model.predict(X_new)

print("✅ 模型保存和加载成功")

---

## 5. Auto-sklearn vs sklearn 对比

### 5.1 sklearn 传统方法

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import time

# sklearn 传统方法
start_time = time.time()

# 定义模型和参数空间
rf = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10]
}

# 网格搜索
grid_search = GridSearchCV(rf, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

sklearn_time = time.time() - start_time
sklearn_score = accuracy_score(y_test, grid_search.predict(X_test))

print(f"sklearn方法:")
print(f"  耗时: {sklearn_time:.2f}秒")
print(f"  准确率: {sklearn_score:.4f}")

### 5.2 Auto-sklearn 方法

In [ ]:
# Auto-sklearn 方法
start_time = time.time()

automl_compare = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    seed=42
)
automl_compare.fit(X_train, y_train)

automl_time = time.time() - start_time
automl_score = accuracy_score(y_test, automl_compare.predict(X_test))

print(f"\nAuto-sklearn方法:")
print(f"  耗时: {automl_time:.2f}秒")
print(f"  准确率: {automl_score:.4f}")

---

## 6. 总结与最佳实践

### Auto-sklearn 核心优势

1. **贝叶斯优化**: 比网格搜索更高效
2. **元学习**: 利用历史经验快速找到好的起点
3. **自动集成**: 自动构建ensemble提升性能
4. **sklearn兼容**: 无缝集成sklearn生态

### 使用建议

```python
# 快速验证（1-3分钟）
automl = AutoSklearnClassifier(time_left_for_this_task=180)

# 标准训练（10-30分钟）
automl = AutoSklearnClassifier(time_left_for_this_task=1800)

# 深度搜索（1-2小时）
automl = AutoSklearnClassifier(time_left_for_this_task=7200)
```

### 优势与局限

**优势**：
- ✅ 智能搜索，效率高于网格搜索
- ✅ 自动集成，性能稳定
- ✅ sklearn兼容性好

**局限**：
- ❌ 依赖历史数据集（元学习）
- ❌ 搜索时间较长
- ❌ Windows支持不完善

---

## 🎯 练习任务

1. **任务1**: 对比 Auto-sklearn 和手动调参的效果
2. **任务2**: 理解贝叶斯优化的搜索过程
3. **任务3**: 分析 Auto-sklearn 的集成策略
4. **任务4**: 在自己的数据集上使用 Auto-sklearn

---

## 📚 参考资源

- [Auto-sklearn 官方文档](https://automl.github.io/auto-sklearn/)
- [贝叶斯优化论文](https://arxiv.org/abs/1206.2944)
- [元学习论文](https://arxiv.org/abs/1802.03268)

---

**下一节**: `03_flaml_optimization.ipynb` - FLAML 优化技巧